In [3]:
from pathlib import Path
out_path_parquet = "/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet"
df = con.execute(""" Select * from read_parquet(?) """,[str(out_path_parquet)]).df()
df.head()

,year,country_of_asylum,country_of_origin,refugees,returned_refugees,asylum_seekers,idps,returned_idps,stateless,hst,ooc,date
0,2020,AFG,AFG,0,0,0,2886317,0,0,0,82595,2025-09-04
1,2020,ALB,AFG,0,0,0,0,0,0,0,5,2025-09-04
2,2020,EGY,AFG,34,0,44,0,0,0,0,0,2025-09-04
3,2020,ARG,AFG,12,0,0,0,0,0,0,0,2025-09-04
4,2020,ARM,AFG,5,0,0,0,0,0,0,0,2025-09-04


In [9]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
from dis import disco
import os
import glob
from pathlib import Path
path = Path(r"/home/faacosta0245695/conflit/conflit_warehouse/data/bronze")
def discovery (path_warehouse: Path, chunk_size=8192):
    for root, dirs, files in os.walk(path_warehouse):
        for file in files:
            if file.endswith('.parquet'):
                print(root, file)
discovery(path)

/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04 [] part-000.parquet


In [45]:
import os
import hashlib
from pathlib import Path
from datetime import datetime
def compute_file_hash(path: Path, chunk_size=65536) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while chunk := f.read(chunk_size):
            h.update(chunk)
    return h.hexdigest()

def discovery(path_warehouse: Path, chunk_size=65536):
    records = []
    for root, _, files in os.walk(path_warehouse):
        for file in files:
            if file.endswith(".parquet"):
                file_path = Path(root) / file
                records.append({
                    "file_path": str(file_path),
                    "content_hash": compute_file_hash(file_path, chunk_size),
                    "file_size": file_path.stat().st_size,
                    "discovered_at": datetime.now(),
                    "source_partition": next((p for p in file_path.parts if p.startswith("date=")), None)
                })
    return records

# Example usage
bronze_path = Path("/home/faacosta0245695/conflit/conflit_warehouse/data/bronze")
files = discovery(bronze_path)
print(files[:2])  # peek first two

[{'file_path': '/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet', 'content_hash': '6ae7cf49a930577cc4a3f216f2e09539fe1b9e7eb7ac0731b820ebc8189acd8a', 'file_size': 31338, 'discovered_at': datetime.datetime(2025, 9, 8, 15, 54, 59, 285006), 'source_partition': 'date=2025-09-04'}]
